In [1]:
import geopandas as gpd

In [5]:
pplants = gpd.read_file(r'.\data\PowerPlants_US_EIA\PowerPlants_US_202001.shp')

In [13]:
pplants['combine'] = pplants[['Geo_MW'and "test",'Bat_MW','Other_MW']].values.tolist()

KeyError: "['test'] not in index"

In [9]:
pplants

,Plant_Code,Plant_Name,Utility_Na,Utility_ID,sector_nam,City,County,StateName,Zip,Street_Add,...,Bat_MW,Other_MW,source_des,tech_desc,Source,Period,Latitude,Longitude,geometry,combine
0,2,Bankhead Dam,Alabama Power Co,195,Electric Utility,Northport,Tuscaloosa,Alabama,35476,19001 Lock 17 Road,...,None,None,Hydroelectric = 53 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",202001,33.458665,-87.356823,POINT (-87.35682 33.45867),"[None, None, None]"
1,3,Barry,Alabama Power Co,195,Electric Utility,Bucks,Mobile,Alabama,36512,North Highway 43,...,None,None,"Coal = 1118.5 MW, Natural Gas = 1268.4 MW",Conventional Steam Coal; Natural Gas Fired Com...,"EIA-860, EIA-860M and EIA-923",202001,31.006900,-88.010300,POINT (-88.01030 31.00690),"[None, None, None]"
2,4,Walter Bouldin Dam,Alabama Power Co,195,Electric Utility,Wetumpka,Elmore,Alabama,36092,750 Bouldin Dam Road,...,None,None,Hydroelectric = 224.1 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",202001,32.583889,-86.283056,POINT (-86.28306 32.58389),"[None, None, None]"
3,7,Gadsden,Alabama Power Co,195,Electric Utility,Gadsden,Etowah,Alabama,35903,1000 Goodyear Ave,...,None,None,Natural Gas = 130 MW,Natural Gas Steam Turbine,"EIA-860, EIA-860M and EIA-923",202001,34.012800,-85.970800,POINT (-85.97080 34.01280),"[None, None, None]"
4,9,Copper,El Paso Electric Co,5701,Electric Utility,El Paso,El Paso,Texas,79915,651 Hawkins Blvd.,...,None,None,Natural Gas = 63 MW,Natural Gas Fired Combustion Turbine,"EIA-860, EIA-860M and EIA-923",202001,31.756900,-106.375000,POINT (-106.37500 31.75690),"[None, None, None]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9763,63514,Moonlight Packing - Phase 2,CalCom Energy,63226,IPP Non-CHP,None,None,California,0,None,...,None,None,Solar = 1.4 MW,Solar Photovoltaic,"EIA-860, EIA-860M and EIA-923",202001,36.585794,-119.469986,POINT (-119.46999 36.58579),"[None, None, None]"
9764,63516,California City,GSRP,61944,IPP Non-CHP,None,None,California,0,None,...,None,None,Solar = 1.9 MW,Solar Photovoltaic,"EIA-860, EIA-860M and EIA-923",202001,35.157000,-117.981000,POINT (-117.98100 35.15700),"[None, None, None]"
9765,63523,Spruce - WCMD - Rubble II,GSRP,61944,IPP Non-CHP,None,None,Maryland,0,None,...,None,None,Solar = 2 MW,Solar Photovoltaic,"EIA-860, EIA-860M and EIA-923",202001,39.621430,-77.825430,POINT (-77.82543 39.62143),"[None, None, None]"
9766,63524,Spruce - WCMD - Rubble I,GSRP,61944,IPP Non-CHP,None,None,Maryland,0,None,...,None,None,Solar = 2 MW,Solar Photovoltaic,"EIA-860, EIA-860M and EIA-923",202001,39.618490,-77.823320,POINT (-77.82332 39.61849),"[None, None, None]"
